In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'n2o': 3.2e-07}
band = [9]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0]
  ng_refs = [7]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.8)]
  wgt = [(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9)]
o n2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [2, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.8, 1.66), (1.6, 1.8)]
  wgt = [(0.6, 0.5), (0.6, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.033703                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.008569
0.000750      2.0 -0.033397                  2.0 -0.008406
0.001052      3.0 -0.032742                  3.0 -0.007974
0.001476      4.0 -0.032066                  4.0 -0.007569
0.002070      5.0 -0.031380                  5.0 -0.007153
0.002904      6.0 -0.030690                  6.0 -0.006728
0.004074      7.0 -0.030030                  7.0 -0.006296
0.005714      8.0 -0.029377                  8.0 -0.005851
0.008015      9.0 -0.028754                  9.0 -0.005402
0.011243     10.0 -0.028176                 10.0 -0.004941
0.015771     11.0 -0.027661                 11.0 -0.004474
0.022122     12.0 -0.027241                 12.0 -0.004001
0.031031     13.0 -0.026961                 13.0 -0.003517
0.043528     14.0 -0.026645                 14.0 -0.002894
0.061057     15.0 -0.023579                 15.0 -0.000492
0.085645     16.0 -0.015531                 16.0  0.005064
0.120136     17.0 -0.003856                 17.0  0.013051
0.168516     18.0  0.010468                 18.0  0.023025
0.236378     19.0  0.027898                 19.0  0.035357
0.331549     20.0  0.049063                 20.0  0.050511
0.465100     21.0  0.074832                 21.0  0.069053
0.652400     22.0  0.102725                 22.0  0.089271
0.915100     23.0  0.109488                 23.0  0.096675
1.283650     24.0  0.093079                 24.0  0.088921
1.800600     25.0  0.071698                 25.0  0.075966
2.525700     26.0  0.049595                 26.0  0.060011
3.542800     27.0  0.032860                 27.0  0.045139
4.969550     28.0  0.021834                 28.0  0.032464
6.970850     29.0  0.014832                 29.0  0.021950
9.778100     30.0  0.010306                 30.0  0.013544
13.715850    31.0  0.007067                 31.0  0.007088
19.239350    32.0  0.004359                 32.0  0.002229
26.987250    33.0  0.001794                 33.0 -0.001641
37.855300    34.0 -0.000882                 34.0 -0.005343
53.100050    35.0 -0.005210                 35.0 -0.010469
73.887500    36.0 -0.010924                 36.0 -0.016705
97.662500    37.0 -0.015779                 37.0 -0.021646
121.437500   38.0 -0.017892                 38.0 -0.023756
145.212500   39.0 -0.017662                 39.0 -0.023658
168.987500   40.0 -0.017270                 40.0 -0.022998
192.762500   41.0 -0.016436                 41.0 -0.021758
216.537500   42.0 -0.014633                 42.0 -0.019487
240.312500   43.0 -0.011451                 43.0 -0.016088
264.087500   44.0 -0.006801                 44.0 -0.011525
287.862500   45.0 -0.000635                 45.0 -0.004405
311.637500   46.0  0.007089                 46.0  0.004196
335.412500   47.0  0.016314                 47.0  0.012489
359.187500   48.0  0.026448                 48.0  0.021796
382.962500   49.0  0.035890                 49.0  0.032223
406.737500   50.0  0.047021                 50.0  0.044014
430.512500   51.0  0.059379                 51.0  0.055433
454.287500   52.0  0.072623                 52.0  0.067046
478.062500   53.0  0.086410                 53.0  0.080352
501.837500   54.0  0.100260                 54.0  0.095977
525.612500   55.0  0.113988                 55.0  0.112543
549.387500   56.0  0.127114                 56.0  0.127909
573.162500   57.0  0.139496                 57.0  0.140640
596.937500   58.0  0.151219                 58.0  0.151023
620.712500   59.0  0.162402                 59.0  0.160498
644.487500   60.0  0.173787                 60.0  0.171399
668.262500   61.0  0.185593                 61.0  0.184769
692.037500   62.0  0.196095                 62.0  0.198713
715.812500   63.0  0.201206                 63.0  0.208222
739.587500   64.0  0.206297                 64.0  0.218928
76

# Fluxes by Layer

CRD                           CLIRAD                \
                         flug       fldg      fnetg       flug          fldg   
pressure     level                                                             
0.000000e+00 1     -15.217054   0.000000 -15.217054        NaN           NaN   
1.000000e-08 1            NaN        NaN        NaN -15.481725  9.044474e-08   
6.244000e-04 2     -15.217057   0.000001 -15.217056 -15.481727  6.865196e-07   
8.759000e-04 3     -15.217058   0.000001 -15.217057 -15.481727  9.306154e-07   
1.228600e-03 4     -15.217060   0.000002 -15.217058 -15.481728  1.282310e-06   
1.723400e-03 5     -15.217063   0.000003 -15.217060 -15.481729  1.787319e-06   
2.417400e-03 6     -15.217067   0.000004 -15.217063 -15.481730  2.512692e-06   
3.390900e-03 7     -15.217072   0.000006 -15.217066 -15.481732  3.554986e-06   
4.756500e-03 8     -15.217080   0.000009 -15.217071 -15.481734  5.052719e-06   
6.672000e-03 9     -15.217090   0.000012 -15.217078 -15.481738  7.205491e-06   
9.358900e-03 10    -15.217105   0.000018 -15.217087 -15.481743  1.029914e-05   
1.312780e-02 11    -15.217125   0.000025 -15.217100 -15.481749  1.474565e-05   
1.841450e-02 12    -15.217153   0.000036 -15.217117 -15.481759  2.113558e-05   
2.583020e-02 13    -15.217193   0.000052 -15.217141 -15.481771  3.031588e-05   
3.623230e-02 14    -15.217248   0.000074 -15.217174 -15.481789  4.350625e-05   
5.082340e-02 15    -15.217326   0.000106 -15.217220 -15.481813  6.258033e-05   
7.129060e-02 16    -15.217432   0.000155 -15.217277 -15.481844  9.238710e-05   
1.000000e-01 17    -15.217569   0.000238 -15.217330 -15.481878  1.440058e-04   
1.402710e-01 18    -15.217734   0.000386 -15.217349 -15.481908  2.360464e-04   
1.967600e-01 19    -15.217921   0.000643 -15.217279 -15.481917  3.992908e-04   
2.759970e-01 20    -15.218101   0.001085 -15.217017 -15.481873  6.869641e-04   
3.871000e-01 21    -15.218208   0.001837 -15.216371 -15.481712  1.190314e-03   
5.431000e-01 22    -15.218090   0.003102 -15.214988 -15.481312  2.066562e-03   
7.617000e-01 23    -15.217476   0.005148 -15.212328 -15.480481  3.547599e-03   
1.068500e+00 24    -15.216292   0.007943 -15.208349 -15.479152  5.732987e-03   
1.498800e+00 25    -15.214720   0.011115 -15.203605 -15.477426  8.539783e-03   
2.102400e+00 26    -15.212855   0.014377 -15.198478 -15.475362  1.190781e-02   
2.949000e+00 27    -15.210889   0.017385 -15.193504 -15.473099  1.566345e-02   
4.136600e+00 28    -15.208920   0.020038 -15.188882 -15.470731  1.964615e-02   
5.802500e+00 29    -15.207002   0.022429 -15.184573 -15.468349  2.367137e-02   
8.139200e+00 30    -15.205224   0.024756 -15.180468 -15.466116  2.751466e-02   
1.141700e+01 31    -15.203750   0.027284 -15.176466 -15.464319  3.097732e-02   
1.601470e+01 32    -15.202884   0.030267 -15.172617 -15.463502  3.402110e-02   
2.246400e+01 33    -15.203187   0.033900 -15.169287 -15.464658  3.687964e-02   
3.151050e+01 34    -15.205656   0.038292 -15.167364 -15.469631  4.009335e-02   
4.420010e+01 35    -15.212082   0.043393 -15.168689 -15.481813  4.424254e-02   
6.200000e+01 36    -15.227565   0.047889 -15.179676 -15.508440  4.879423e-02   
8.577500e+01 37    -15.260952   0.050512 -15.210440 -15.559257  5.255933e-02   
1.095500e+02 38    -15.307813   0.052934 -15.254880 -15.623478  5.581265e-02   
1.333250e+02 39    -15.365100   0.059832 -15.305268 -15.696489  6.191192e-02   
1.571000e+02 40    -15.428976   0.073965 -15.355011 -15.774616  7.340398e-02   
1.808750e+02 41    -15.498516   0.094867 -15.403648 -15.856656  9.066678e-02   
2.046500e+02 42    -15.573711   0.123774 -15.449937 -15.942427  1.151537e-01   
2.284250e+02 43    -15.657280   0.166131 -15.491149 -16.034512  1.523503e-01   
2.522000e+02 44    -15.751265   0.227866 -15.523399 -16.134825  2.073496e-01   
2.759750e+02 45    -15.858372   0.315820 -15.542552 -16.245885  2.859497e-01   
2.997500e+02 46    -15.981729   0.437388 -15.544341 -16.370582  3.982400e-01   
3.235250e+02 47    -16.124

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')